In [ ]:
import requests
import pandas as pd

api_url = "https://ows.goszakup.gov.kz/v3/refs/ref_fkrb_subprogram"
token = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
file = "fkrb_subprogram_data.csv"

def retrieve_fkrb_subprogram_data():
    """Получает данные справочника подпрограмм ФКР через API и сохраняет в CSV."""
    subprogram_list = []
    search_after = None

    while True:
        try:
            params = {"limit": 500}
            if search_after:
                params["page"] = "next"
                params["search_after"] = search_after

            response = requests.get(api_url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            items = data.get("items", [])
            if not items:
                print("Все данные загружены.")
                break

            for item in items:
                subprogram_list.append({
                    "id": item.get("id"),
                    "ppr": item.get("ppr"),
                    "name_kz": item.get("name_kz"),
                    "name_ru": item.get("name_ru")
                })

            print(f"Текущая загрузка: {len(subprogram_list)} записей")
            search_after = items[-1].get("id")
            if not data.get("next_page"):
                break

        except requests.exceptions.RequestException as e:
            print(f"Ошибка: {e}")
            break

    export_to_csv(subprogram_list)

def export_to_csv(dataset):
    """Сохраняет данные в CSV."""
    if not dataset:
        print("Нет данных для сохранения.")
        return
    df = pd.DataFrame(dataset)
    df.to_csv(file, index=False, encoding="utf-8-sig", sep="|", quotechar="'", escapechar="\\")
    print(f"Данные успешно сохранены в файл: {file} ({len(dataset)} записей)")

retrieve_fkrb_subprogram_data()